<a href="https://colab.research.google.com/github/saksham751/Neural-Network-from-Scratch/blob/main/neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.mnist import load_data
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward_propagation(self, input):
        raise NotImplementedError
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [3]:
class complateLayer(Layer):
  
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        #print(output_error.shape, self.weights.T.shape)
        #print(self.input.T.shape, output_error.shape)
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T.reshape(self.input.T.shape[0],1), output_error)
        
        self.weights -= learning_rate * weights_error
        self.bias = self.bias-learning_rate * output_error
        return input_error

In [4]:

class activationFunction(Layer):
    def __init__(self, activation, activationDer):
        self.activation = activation
        self.activationDer = activationDer

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activationDer(self.input) * output_error

In [5]:
    def sig(x):
      return 1/(1 + np.exp(-x))

    def sig_der(x):
      return sig(x)*(1-sig(x))
  
    def tanh(x):
      return np.tanh(x)

    def tanh_prime(x):
      return 1-np.tanh(x)**2

    def relu(x):
      return np.maximum(x, 0)

    def relu_der(x):
      return 1. * (x > 0)

    def linear(x):
      return x

    def linear_der(x):
      return 1

    

In [6]:

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mseDer(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):

        samples = len(input_data)
        result = []
        for i in range(samples):

            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate):

        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
  
                output = x_train[j]
                
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)

                error = self.loss_prime(y_train[j], output)
                #error.reshape(784,1)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
                #print('error=%f'%(err/j))

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [8]:
#(x_train, y_train), (x_test, y_test)= load_data( path="mnist.npz" )
iris = datasets.load_iris()
x, y = iris.data, iris.target

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [10]:
x_train.shape[1]
classes=np.unique(y_train).size
y_train

array([2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 0,
       0, 1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1, 2, 0,
       2, 1, 1, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1,
       0, 0, 1, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0,
       0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2,
       1, 1, 2, 2, 1, 0, 1, 1, 2, 2])

In [11]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_train.shape

(120, 3)

In [12]:
y_train

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0

In [13]:
network1 = Network()
x1=x_train.shape[1]
x2=4
network1.add(complateLayer(x1, x2))
network1.add(activationFunction(tanh, tanh_prime))
network1.add(complateLayer(x2, classes))
network1.add(activationFunction(tanh, tanh_prime))


In [14]:
# x_train.reshape(x_train.shape[0],1,784)
# x_new2=x_train.reshape(x_train.shape[0],1,784)
# x_new=np.delete(x_new2,np.s_[0:50000],axis=0)
# x_new.shape

In [15]:
# y_new2=y_train.reshape(y_train.shape[0],1)
# y_new=np.delete(y_new2,np.s_[0:50000],axis=0)
# y_new.shape


In [16]:
x_train.shape

(120, 4)

In [17]:
network1.use(mse, mseDer)

network1.fit(x_train ,y_train, epochs=50, learning_rate=0.005)

epoch 1/50   error=0.645912
epoch 2/50   error=0.643462
epoch 3/50   error=0.640311
epoch 4/50   error=0.636103
epoch 5/50   error=0.630207
epoch 6/50   error=0.621405
epoch 7/50   error=0.607183
epoch 8/50   error=0.582652
epoch 9/50   error=0.544831
epoch 10/50   error=0.511267
epoch 11/50   error=0.492385
epoch 12/50   error=0.469493
epoch 13/50   error=0.427924
epoch 14/50   error=0.384973
epoch 15/50   error=0.373240
epoch 16/50   error=0.372642
epoch 17/50   error=0.372644
epoch 18/50   error=0.372589
epoch 19/50   error=0.372511
epoch 20/50   error=0.372424
epoch 21/50   error=0.372330
epoch 22/50   error=0.372230
epoch 23/50   error=0.372123
epoch 24/50   error=0.372008
epoch 25/50   error=0.371885
epoch 26/50   error=0.371752
epoch 27/50   error=0.371608
epoch 28/50   error=0.371453
epoch 29/50   error=0.371283
epoch 30/50   error=0.371098
epoch 31/50   error=0.370895
epoch 32/50   error=0.370672
epoch 33/50   error=0.370425
epoch 34/50   error=0.370150
epoch 35/50   error=0.3

In [18]:
out = network1.predict(x_test)
print(out)

[array([[0.28345328, 0.38599028, 0.87554577]]), array([[0.28345194, 0.38602281, 0.87554642]]), array([[0.28345189, 0.38602408, 0.87554645]]), array([[0.28345687, 0.38590418, 0.87554398]]), array([[0.28367226, 0.38034202, 0.87541821]]), array([[0.28345306, 0.38599571, 0.87554587]]), array([[0.28345516, 0.38594472, 0.87554486]]), array([[0.28369723, 0.37947749, 0.87538192]]), array([[0.28368327, 0.379961  , 0.8753944 ]]), array([[0.283456  , 0.38592574, 0.87554441]]), array([[0.28345209, 0.38601929, 0.87554635]]), array([[0.28363229, 0.38150863, 0.87545199]]), array([[0.28345827, 0.38587186, 0.87554321]]), array([[0.28345213, 0.38601824, 0.87554633]]), array([[0.2834516 , 0.38603113, 0.87554659]]), array([[0.28345178, 0.38602674, 0.8755465 ]]), array([[0.28366356, 0.38072278, 0.87543539]]), array([[0.28376762, 0.377052  , 0.87531062]]), array([[0.28345391, 0.38597514, 0.87554546]]), array([[0.28365693, 0.38082788, 0.87542773]]), array([[0.28365357, 0.38094474, 0.87543684]]), array([[0.28

In [19]:
x_train.shape

(120, 4)

In [20]:
db = datasets.load_diabetes()
x, y = db.data, db.target

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [22]:
x_train
y_train

array([170.,  77.,  49.,  66., 144., 113., 233., 162., 121.,  88., 302.,
       128.,  52., 178.,  90., 164., 135., 103., 200., 178., 261.,  64.,
        59.,  79.,  47., 107.,  39., 151., 104., 217., 232.,  55., 245.,
       131.,  64., 222., 249., 128., 293., 138., 273., 158., 168., 103.,
       140., 146., 277., 116., 233.,  94., 295., 191., 241., 154., 141.,
       252., 198., 288., 101., 179., 129., 174.,  91.,  84., 177., 219.,
        67., 113.,  70.,  53., 150.,  40., 258., 168., 283., 252.,  43.,
        74.,  78., 209., 270.,  90.,  73., 310., 102., 115.,  85., 104.,
        68., 101., 139.,  63.,  72.,  58., 131.,  71.,  98.,  88., 277.,
       104., 167., 108., 111., 214., 135., 196., 189.,  77., 225.,  60.,
        67., 292., 143., 281.,  25., 200.,  91., 263.,  95., 144., 232.,
       109.,  51.,  64., 283., 259., 214.,  60., 111., 258.,  68., 174.,
       118., 113., 306.,  70.,  78., 191., 220., 183., 124.,  44.,  63.,
       114., 229.,  77., 168., 262.,  91.,  85., 12

In [23]:
# from keras.utils import np_utils
# y_train = np_utils.to_categorical(y_train)
# y_train.shape

In [24]:
y_train

array([170.,  77.,  49.,  66., 144., 113., 233., 162., 121.,  88., 302.,
       128.,  52., 178.,  90., 164., 135., 103., 200., 178., 261.,  64.,
        59.,  79.,  47., 107.,  39., 151., 104., 217., 232.,  55., 245.,
       131.,  64., 222., 249., 128., 293., 138., 273., 158., 168., 103.,
       140., 146., 277., 116., 233.,  94., 295., 191., 241., 154., 141.,
       252., 198., 288., 101., 179., 129., 174.,  91.,  84., 177., 219.,
        67., 113.,  70.,  53., 150.,  40., 258., 168., 283., 252.,  43.,
        74.,  78., 209., 270.,  90.,  73., 310., 102., 115.,  85., 104.,
        68., 101., 139.,  63.,  72.,  58., 131.,  71.,  98.,  88., 277.,
       104., 167., 108., 111., 214., 135., 196., 189.,  77., 225.,  60.,
        67., 292., 143., 281.,  25., 200.,  91., 263.,  95., 144., 232.,
       109.,  51.,  64., 283., 259., 214.,  60., 111., 258.,  68., 174.,
       118., 113., 306.,  70.,  78., 191., 220., 183., 124.,  44.,  63.,
       114., 229.,  77., 168., 262.,  91.,  85., 12

In [25]:
network1 = Network()
x1=x_train.shape[1]
x2=x1*2
network1.add(complateLayer(x1, x2))
network1.add(activationFunction(relu, relu_der))
network1.add(complateLayer(x2, 1))
network1.add(activationFunction(linear, linear_der))


In [26]:
network1.use(mse, mseDer)

network1.fit(x_train ,y_train, epochs=50, learning_rate=0.001)

epoch 1/50   error=6609.467574
epoch 2/50   error=5278.709389
epoch 3/50   error=4897.286642
epoch 4/50   error=4707.425909
epoch 5/50   error=4588.848188
epoch 6/50   error=4504.196967
epoch 7/50   error=4441.861569
epoch 8/50   error=4385.466929
epoch 9/50   error=4336.421500
epoch 10/50   error=4292.069885
epoch 11/50   error=4240.357981
epoch 12/50   error=4205.995101
epoch 13/50   error=4180.555538
epoch 14/50   error=4132.117388
epoch 15/50   error=4103.847811
epoch 16/50   error=4083.995874
epoch 17/50   error=4043.789953
epoch 18/50   error=4016.587065
epoch 19/50   error=3935.569013
epoch 20/50   error=3890.430001
epoch 21/50   error=3856.396643
epoch 22/50   error=3822.358965
epoch 23/50   error=3785.215547
epoch 24/50   error=3758.260437
epoch 25/50   error=3724.802406
epoch 26/50   error=3698.987481
epoch 27/50   error=3671.308517
epoch 28/50   error=3649.459273
epoch 29/50   error=3668.058454
epoch 30/50   error=3632.846739
epoch 31/50   error=3619.896243
epoch 32/50   err

In [27]:
out = network1.predict(x_test)
print(out)

[array([[117.04438701]]), array([[112.386688]]), array([[170.32316954]]), array([[113.9898358]]), array([[120.01999125]]), array([[115.0666298]]), array([[199.37076638]]), array([[108.4017489]]), array([[106.99268136]]), array([[106.99268136]]), array([[107.04882188]]), array([[134.89375333]]), array([[121.98952225]]), array([[152.56485915]]), array([[172.08124233]]), array([[128.00584771]]), array([[107.0620001]]), array([[108.37533863]]), array([[134.2933391]]), array([[148.0209955]]), array([[118.87381387]]), array([[179.88745642]]), array([[118.60920415]]), array([[106.99268136]]), array([[106.99268136]]), array([[142.19375565]]), array([[106.99268136]]), array([[106.99268136]]), array([[116.54787399]]), array([[129.48799289]]), array([[106.99878233]]), array([[181.23553491]]), array([[142.4435242]]), array([[163.22873989]]), array([[144.19124436]]), array([[106.99268136]]), array([[106.99268136]]), array([[106.99268136]]), array([[168.95198061]]), array([[106.99268136]]), array([[

In [28]:
y_test

array([185., 150., 246., 184., 110., 202., 336.,  69.,  69.,  87.,  66.,
       164., 265., 198., 248., 171., 102., 182., 262., 233., 151., 275.,
       230.,  39.,  42., 122.,  81.,  31., 156., 161.,  80., 215., 310.,
       163., 265.,  84.,  54., 137., 248.,  72., 248.,  59.,  97.,  61.,
        72., 296.,  55.,  75.,  53., 173., 158.,  96., 140., 186.,  93.,
       123., 237., 138., 292.,  78., 182., 127., 321., 109., 212.,  45.,
        96., 109.,  89.,  96.,  60.,  51., 200., 265.,  68.,  91., 172.,
       115., 125.,  53.,  63., 259., 214., 195., 258., 235., 173.,  59.,
       219.])